In [1]:
import random
import re
import math
from collections import OrderedDict

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [2]:
USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

In [3]:
USE_CUDA

True

In [4]:
def pre_processing(file):
    with open(file, 'r', encoding='utf8') as f:
        lines = f.readlines()
    corpus = OrderedDict()
    title_i = -1
    group_id = -1
    line_id = 0
    for i, line in enumerate(lines):
        if line[:4] == '<doc':
            title_i = i + 1
            group_id += 1
            continue
        if i == title_i:
            continue
        for sline in line.splitlines():
            sline = re.sub(r'[^ ㄱ-ㅣ가-힣\.,]+', 'x', sline)
            sline = re.sub(r'다\.', '다.\n', sline)
            slines = sline.split('\n')
            for s in slines:
                s = s.strip()
                if len(s) > 20:
                    if 'x' not in s:
                        corpus[line_id] = {}
                        corpus[line_id][group_id] = s
                        line_id += 1
    return corpus


def print_lines(corpus, n=10):
    for i in range(n):
        try:
            print(corpus[i])
        except ValueError:
            print('not enough lines')

In [5]:
corpus = pre_processing('wiki.txt')

In [6]:
print_lines(corpus)

{0: '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.'}
{0: '조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.'}
{0: '카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.'}
{0: '카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.'}
{0: '그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다.'}
{0: '지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통령 중 하나다.'}
{0: '인권 문제와 주한미군 철수 문제로 한때 한미 관계가 불편하기도 했다.'}
{0: '미국의 빈곤층 지원 활동, 사랑의 집짓기 운동, 국제 분쟁 중재 등의 활동도 했다.'}
{0: '카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 선행하는 행위에 대해 깊은 유감을 표시 하며 미국의 군사적 활동에 강한 반대 입장을 보이고 있다.'}
{0: '특히 국제 분쟁 조정을 위해 북한의 김일성, 아이티의 세드라스 장군, 팔레인스타인의 하마스, 보스니아의 세르비아계 정권 같이 미국 정부에 대해 협상을 거부하면서 사태의 위기를 초래한 인물 및 단체를 직접 만나 분쟁의 원인을 근본적으로 해결하기 위해 힘썼다.'}


In [7]:
with open('corpus.txt', 'w', encoding='utf8') as f:
    for values in corpus.values():
        for line in values.values():
            f.write(f'{line}\n')